# 1.0 Example - Correlating Process Paramters to XAS Spectra Time Steps

Time-dependent process parameters (Mass Spec, LabView) will be interpolated onto the time-grid of the XAS data making for quick spectra-process parameters plotting and lookup

Correlated data is retunred as a pandas dataframe and saved as a .csv file.

### Things to have ready when attemtping this example:
1. Directoty containing only XAS spectra
2. Data structure of the XAS files, key paramters are where the time stamp is stired
3. Path and file name for process paramters files (MS and LV)
4. Directory where to write the summary file

## Step 1. Import Modules

In [1]:
# File Handling
import os

# Math and Data Handling
import pandas as pd

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp

No conda env active, defaulting to base


## Step 2. Path Definitions

Specify paths to XAS data, process data streams, and output files

#### Notes:
1. data_dir can only contain XAS spectra that were collected in the same manner (e.g. steady state scans, or all in-situ scan)
2. *_filename paths must be the entire path with file extension
3. output_dir must be an existing folder
4. result_fname should not have an extension


In [2]:
# Folder containing ONLY CXAS spectra
data_dir = os.path.join(pwd, 'sample data\Raw Data') 

# Mass Spec File:
MS_filename = os.path.join(pwd, 'sample data\SnO2_TPR_MS.csv') 

# AmP LabView File:
LV_filename = os.path.join(pwd, 'sample data\SnO2_TPR_LV.txt')

# Folder where the correlated datatable will be written
output_dir = os.path.join(pwd, 'sample results')

# Name of fole to be written
result_fname = 'SnO2_TPR_Process_Correlation'

## Step 3. Define Beamline Data Structure

Common data formats for SSRL beamlines can be found in the "BL specific XAS data structures.ipynb" notebook.

For this example the key values needed are 'time stamp' (must be true), 'time on line', 'time format', 'padded scan numbers' (padded = leading zeros before the scan number, e.g. *_001 instead of *_1), and 'is QEXAFS'

In [4]:
### Define the layout of the xas data file and how spectra are calcualted
xas_data_structure = {
    'time stamp': True,
    'time on line': 5,
    'time format': '# This Scan Create Date:\t%m/%d/%Y %I:%M:%S %p ',
    'padded scan numbers': True,
    'column names': ['Encoder', 'Energy', 'ADC_01', 'ADC_02', 'ADC_03', 'ADC_04', 'ADC_05', 'ADC_06', 'ADC_07', 'ADC_08'],
    'energy column' : 'Energy', # Energy [eV]
    'sample numerator': 'ADC_01', # I0
    'sample denominator': 'ADC_02', # I1
    'sample ln': True,
    'sample invert': False,
    'reference numerator': 'ADC_02', # I1
    'reference denominator': 'ADC_03', # I2
    'reference ln': True,
    'reference invert': False,
    'is QEXAFS': False
}

## Step 4. Summarize file names and time steps of XAS spectra 

### DO NOT MODIFY THIS SECTION

In [5]:
time_stamp = xas_data_structure['time stamp']
time_line = xas_data_structure['time on line']
time_format = xas_data_structure['time format']
padded = xas_data_structure['padded scan numbers']
is_QEXAFS = xas_data_structure['is QEXAFS']


XAS_summary = xfcts.CXAS_Sorted(data_dir,
                   time_stamp=time_stamp, time_line=time_line, time_format=time_format,
                   padded=padded, is_QEXAFS = is_QEXAFS)

In [6]:
# Visualize the summary fole of the XAS data 
XAS_summary

,TOS [s],File Name,Padded Name,Path
Time,,,,
2021-06-14 16:33:52,0.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:35:37,105.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0002,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0002,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:37:23,211.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0003,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0003,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:39:11,319.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0004,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0004,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:40:57,425.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0005,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0005,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:42:42,530.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0006,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0006,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:44:27,635.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0007,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0007,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:46:13,741.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0008,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0008,C:\Users\ashoff\OneDrive - SLAC National Accel...
2021-06-14 16:47:59,847.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0009,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0009,C:\Users\ashoff\OneDrive - SLAC National Accel...


## Step 5. Import Process Parameter Files 

The catxas.general_functions module has a list of fuctions for reading different process files. More formats can be added as needed

### MODIFY by:
Adding additional lines to read in all process variable streams

In [7]:
MS_data = process.ReadMSData(MS_filename)

LV_data = process.ReadLVData(LV_filename)

## Step 6. Interpolate Process Paramters onto XAS spectra Timestamps

### MODIFY by:
1. Adding or removing functions to account for all process variable streams
2. Update list of dataframes that will be joined in the output file

In [8]:
MS_interp = fcts.mergeindex(XAS_summary, MS_data) 
LV_interp = fcts.mergeindex(XAS_summary, LV_data)

concat_list = [XAS_summary['TOS [s]'],XAS_summary['File Name'], MS_interp, LV_interp]

## Step 7. Concatenate XAS Filename and Time Summar with Process Parameters

### DO NOT MODIFY THIS SECTION

In [9]:
process_params = pd.concat(concat_list, axis = 1)

In [10]:
# Visualize the xas-process parameters summary dataframe 
process_params

,TOS [s],File Name,H2,He,H2O,28,O2,Ar,CO2,T,...,Extra TC3 - Extra TC3 [°C],Extra TC4 - Extra TC4 [°C],Extra TC5 - Extra TC5 [°C],Extra TC6 - Extra TC6 [°C],Extra TC7 - Extra TC7 [°C],Extra TC8 - Extra TC8 [°C],Extra DO 1,Extra DO 2,Extra DO 3,Extra DO 4
Time,,,,,,,,,,,,,,,,,,,,,
2021-06-14 16:33:52,0.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001,4.929417e-10,1.810452e-07,6.951219e-09,8.664734e-09,2.471646e-09,0.000004,7.719114e-11,26.120772,...,2300.190768,2300.464815,2300.464815,2300.464815,2300.343342,2300.343342,0.0,0.0,0.0,0.0
2021-06-14 16:35:37,105.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0002,4.931230e-10,1.827490e-07,6.926960e-09,8.628440e-09,2.426090e-09,0.000004,7.898010e-11,25.907800,...,2300.228154,2300.499174,2300.499174,2300.499174,2300.378966,2300.378966,0.0,0.0,0.0,0.0
2021-06-14 16:37:23,211.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0003,4.713019e-10,1.821100e-07,6.798056e-09,8.669651e-09,2.461149e-09,0.000004,7.286727e-11,26.067593,...,2300.241883,2300.510800,2300.510800,2300.510800,2300.391108,2300.391108,0.0,0.0,0.0,0.0
2021-06-14 16:39:11,319.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0004,4.056493e-09,1.949101e-07,7.054554e-09,9.396570e-09,1.122279e-08,0.000004,1.175604e-10,26.068597,...,2300.262552,2300.529460,2300.529460,2300.529460,2300.409950,2300.409950,0.0,0.0,0.0,0.0
2021-06-14 16:40:57,425.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0005,1.128710e-08,1.397490e-08,7.368780e-09,8.759550e-09,4.797820e-09,0.000004,9.577160e-11,26.015300,...,2300.294947,2300.559966,2300.559966,2300.559966,2300.440447,2300.440447,0.0,0.0,0.0,0.0
2021-06-14 16:42:42,530.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0006,1.628722e-08,7.763893e-09,7.012374e-09,8.508112e-09,3.144702e-09,0.000004,1.011230e-10,25.964397,...,2300.302634,2300.566300,2300.566300,2300.566300,2300.444244,2300.444244,0.0,0.0,0.0,0.0
2021-06-14 16:44:27,635.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0007,1.677209e-07,6.677977e-09,6.732073e-09,8.423802e-09,2.903965e-09,0.000004,9.577366e-11,26.007782,...,2300.321962,2300.584995,2300.584995,2300.584995,2300.463512,2300.463512,0.0,0.0,0.0,0.0
2021-06-14 16:46:13,741.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0008,5.390002e-07,6.013107e-09,6.438386e-09,8.409555e-09,2.787266e-09,0.000004,8.044948e-11,25.961600,...,2300.348302,2300.611476,2300.611476,2300.611476,2300.491137,2300.491137,0.0,0.0,0.0,0.0
2021-06-14 16:47:59,847.0,20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0009,5.453353e-07,5.441864e-09,6.127758e-09,8.365563e-09,2.705257e-09,0.000004,8.813759e-11,26.992001,...,2300.356767,2300.619987,2300.619987,2300.619987,2300.500101,2300.500101,0.0,0.0,0.0,0.0


## Step 8. Save Dataframe to CSV file

### DO NOT MODIFY THIS SECTION

In [11]:
process_params.to_csv(os.path.join(output_dir, result_fname + '.csv'), sep=',', 
                      na_rep='', header=True, index=True, index_label='Time')